# (E08)8th_Project-Movie_Recommend
********************************************************************************

# 1. 데이터 전처리

      유저ID, 영화ID, 평가, 타임스탬프가 기록된 데이터셋을 불러옵니다.
      그 후에 사용하는 앞의 세가지의 컬럼만 남기고, 평가 3점 이상의 평가만 남겨줍니다.

In [1]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'movie_id', 'rating']
ratings = ratings[using_cols]
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [5]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

      영화 제목을 불러오고 영화ID의 제목을 확인해줍니다. Index의 0이 영화ID의 1번입니다. 해당 파일을 불러올 때, 인코딩 에러가 발생하니 적당한 인코딩을 추가해줍니다. 다른 인코딩을 해줘도 정상 작동되는 것을 확인했습니다.

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, encoding = 'ISO-8859-1', sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


# 2. 데이터 확인하기

      전처리를 완료한 ratings에 남아있는 유니크한 유저ID의 수와 영화ID의 수를 확인해줍니다.

In [7]:
# 유저 수
ratings['user_id'].nunique()

6039

In [8]:
# 영화 수
ratings['movie_id'].nunique()

3628

      인기 많은 영화를 30개 확인해봅니다. 영화ID로 나와서 제목을 확인하기 힘듭니다.

In [9]:
# 인기 많은 영화
ratings_count = ratings.groupby('movie_id')['user_id'].count()
ratings_count = ratings_count.sort_values(ascending=False).head(30)

ratings_count

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

# 3. 선호하는 영화 5가지 RATING에 추가하기

      좋아하는 영화 5가지를 선택해줍니다. 저는 결과를 쉽게 확인해주기 위해서 SF계열의 실제로 본 영화를 추가해줬습니다. 그리고 전부 명작입니다. 당연히 5점을 줍니다.

In [10]:
# 본인이 좋아하시는 영화 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = [316, 260, 1196, 1210, 1374]
# 316 = Stargate (1994)
# 260 = Star Wars: Episode IV - A New Hope (1977)
# 1196 = Star Wars: Episode V - The Empire Strikes Back (1980)
# 1210 = Star Wars: Episode VI - Return of the Jedi (1983)
# 1374 = Star Trek: The Wrath of Khan (1982)

# 'mine'이라는 user_id가 위 영화의 평가를 5로 다 주었다고 가정하겠습니다.
my_ratings = pd.DataFrame({'user_id': ['mine']*5, 'movie_id': my_favorite, 'count':[5]*5})

if not ratings.isin({'user_id':['mine']})['user_id'].any():  # user_id에 'mine'이라는 데이터가 없다면
    ratings = ratings.append(my_ratings)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,mine,316,5
1,mine,260,5
2,mine,1196,5
3,mine,1210,5
4,mine,1374,5


      새로 추가된 'mine'을 숫자형으로 바꿔줍니다. ratings를 불러와 잘 바뀌었는지 확인해줍니다. 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 합니다.

In [11]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}

ratings['user_id'] =  ratings['user_id'].map(user_to_idx.get).dropna()

# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['mine'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 합니다. 

ratings.tail(10) 

6039


,user_id,movie_id,count
1000203,6038,1090,3
1000205,6038,1094,5
1000206,6038,562,5
1000207,6038,1096,4
1000208,6038,1097,4
0,6039,316,5
1,6039,260,5
2,6039,1196,5
3,6039,1210,5
4,6039,1374,5


      인덱스도 새로 정렬해줍니다. 비어있는 값을 제거하고 압축합니다.

In [12]:
ratings.reset_index(drop=True, inplace=True)

ratings

,user_id,movie_id,count
0,0,1193,5
1,0,661,3
2,0,914,3
3,0,3408,4
4,0,2355,5
...,...,...,...
836478,6039,316,5
836479,6039,260,5
836480,6039,1196,5
836481,6039,1210,5


      해당 프로젝트를 진행하던 도중 영화ID를 IDX로, 제목을 영화ID로 변경하는데 불편함을 느껴서 영화ID를 기준으로 ratings와 movies를 merge를 이용해 두 데이터를 합쳐줬습니다.

In [13]:
movies_ratings = pd.merge(ratings, movies, on = 'movie_id', how = 'left')
movies_ratings

,user_id,movie_id,count,title,genre
0,0,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,0,661,3,James and the Giant Peach (1996),Animation|Children's|Musical
2,0,914,3,My Fair Lady (1964),Musical|Romance
3,0,3408,4,Erin Brockovich (2000),Drama
4,0,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...
836478,6039,316,5,Stargate (1994),Action|Adventure|Sci-Fi
836479,6039,260,5,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
836480,6039,1196,5,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
836481,6039,1210,5,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War


      마찬가지로 사용하게 될 컬럼만 남겨줍니다.

In [14]:
using_cols = ['user_id', 'title', 'count']
movies_ratings = movies_ratings[using_cols]
movies_ratings

,user_id,title,count
0,0,One Flew Over the Cuckoo's Nest (1975),5
1,0,James and the Giant Peach (1996),3
2,0,My Fair Lady (1964),3
3,0,Erin Brockovich (2000),4
4,0,"Bug's Life, A (1998)",5
...,...,...,...
836478,6039,Stargate (1994),5
836479,6039,Star Wars: Episode IV - A New Hope (1977),5
836480,6039,Star Wars: Episode V - The Empire Strikes Back...,5
836481,6039,Star Wars: Episode VI - Return of the Jedi (1983),5


      위에서 확인했던 인기많은 영화 30개를 확인해봅니다. 대부분 다 아는 영화들인것이 정확한 평가인거같습니다.

In [15]:
# 인기 많은 영화
movie_count = movies_ratings.groupby('title')['user_id'].count()
movie_count = movie_count.sort_values(ascending=False).head(30)

movie_count

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2911
Star Wars: Episode V - The Empire Strikes Back (1980)    2886
Star Wars: Episode VI - Return of the Jedi (1983)        2717
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

      CSR로 만들기 위해서 제목 컬럼을 IDX로 변환해줍니다. 영화제목을 IDX로 변환하는 코드를 작성하고 확인한 후에 IDX를 영화제목으로 변환해주는 코드를 작성했습니다.

In [16]:
movie_unique = movies_ratings['title'].unique()
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}
movies_ratings['title'] =  movies_ratings['title'].map(movie_to_idx.get).dropna()

#movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

/home/aiffel-dj39/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


      확인 결과 둘 다 정상적으로 작동합니다.

In [17]:
movie_to_idx['Stargate (1994)']

544

In [18]:
idx_to_movie[544]

'Stargate (1994)'

      본인이 추가한 평가들이 잘 들어갔는지 마지막으로 확인해줍니다. 모든 것이 잘 변환되어있는 것을 확인했습니다.

In [19]:
# user_id 컬럼을 선택합니다.
# 컬럼의 값과 조건을 비교합니다.
# 그 결과를 새로운 변수에 할당합니다.
is_user = movies_ratings['user_id'] == 6039

# 조건를 충족하는 데이터를 필터링하여 새로운 변수에 저장합니다.
user = movies_ratings[is_user]

# 결과를 출력합니다.
user

,user_id,title,count
836478,6039,544,5
836479,6039,44,5
836480,6039,117,5
836481,6039,64,5
836482,6039,583,5


# 4. CSR Matrix 만들기
     
     이제 유저ID 컬럼과 IDX로 바뀐 제목 컬럼을 이용하여 CSR 매트릭스를 만들어줍니다. 만들 때 숫자형 말고 다른 것이 있으면 컴파일 에러가 발생하는것 같습니다. 생성 결과 6040X3628의 행렬이 만들어진것을 확인했습니다.

In [20]:
from scipy.sparse import csr_matrix

num_user = movies_ratings['user_id'].nunique()
num_movie = movies_ratings['title'].nunique()

print(num_user, num_movie)

csr_data = csr_matrix((movies_ratings['count'], (movies_ratings.user_id, movies_ratings.title)), shape= (num_user, num_movie))
csr_data

6040 3628


<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5. 모델 구성하고 학습시키기
    
      예제에서 진행했듯이 모델을 구성해주고 위에서 만든 CSR로 학습을 진행합니다. 학습을 마친 후 본인의 성향 벡터와 영화의 벡터를 이용하여 정확도를 예상해봅니다.

In [21]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [22]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [23]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [24]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

# 6. 모델이 예측한 선호도 확인하기

In [25]:
mine, stargate = user_to_idx['mine'], movie_to_idx['Stargate (1994)']
mine_vector, stargate_vector = als_model.user_factors[mine], als_model.item_factors[stargate]

print(mine_vector)
print(stargate_vector)

[ 4.3420565e-01 -2.3691906e-01  8.3487517e-01  1.9792056e-02
 -6.7425931e-01 -2.5232890e-01 -4.0742439e-01 -1.1378595e+00
  3.9833495e-01  9.3949959e-02  5.1261121e-01  1.4973220e+00
 -2.2827855e-01  7.7114201e-01  8.3384126e-01 -4.8654530e-02
 -8.1862777e-01 -1.3176940e-01  1.0035108e-01 -7.9671949e-02
 -7.4904019e-01  1.7581974e-01  4.3770096e-01 -2.1579958e-01
 -2.5387332e-01  1.3894135e-01  4.7609117e-02 -3.0102131e-01
  1.8675046e-01  8.5462314e-01 -1.2766261e-01 -4.1149551e-01
 -9.4283052e-02 -2.4488817e-01  6.8272561e-01 -3.3583811e-01
  3.0438483e-01  4.1226365e-02 -1.2439046e-01 -4.6201637e-01
  8.1316453e-01 -9.5242836e-02 -5.9691465e-01 -2.4275973e-01
  4.1264129e-01  8.6374432e-01 -5.1022172e-01 -6.4703636e-02
 -1.3021016e-01  2.6077336e-01 -8.3317667e-02 -1.0635823e+00
 -8.6758268e-01 -3.6377954e-01  8.5451663e-01  5.7612479e-01
  7.3149574e-01 -2.7742168e-01 -1.5694505e-04 -2.7213314e-01
  1.7152543e-01  1.2703879e-01  1.1761250e+00  1.1982466e+00
  5.2909166e-01 -5.74684

In [26]:
np.dot(mine_vector, stargate_vector)

0.2757695

In [27]:
star_wars_I = movie_to_idx['Star Wars: Episode I - The Phantom Menace (1999)'] #Star Wars: Episode I - The Phantom Menace (1999)
star_wars_I_vector = als_model.item_factors[star_wars_I]
np.dot(mine_vector, star_wars_I_vector)

0.4765836

      스타게이트의 경우 5점을 주었음에도 불구하고 0.29밖에 안됬고 스타워즈1은 4,5,6을 평가했는데 0.49로 매우 높게 나왔습니다.
      
      스타트렉:퍼스트 컨택트의 유사한 영화를 찾아봤습니다. 일단 1로 나온것은 자기자신이고 나머지는 IDX로 나와서 잘 모르겠으니 확인해줍니다.

# 7. 내가 좋아하는 영화와 비슷한 영화 추천받기

In [28]:
favorite_movie = 'Star Trek: First Contact (1996)' # Star Trek: First Contact (1996)
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=16) # 16을 준 이유는 자기 자신이 1위를 지니기 때문입니다.

similar_movie

[(471, 0.99999994),
 (113, 0.9025383),
 (558, 0.88010925),
 (699, 0.8279797),
 (586, 0.7756079),
 (584, 0.70790344),
 (1164, 0.65918434),
 (583, 0.64518994),
 (581, 0.6339021),
 (544, 0.5946121),
 (145, 0.50578594),
 (713, 0.46366918),
 (228, 0.43473464),
 (62, 0.42972943),
 (3589, 0.41953653),
 (479, 0.41886035)]

      변환해 주는 코드를 작성한 후 확인해주니 확실히 SF류 영화가 가깝다는 것을 확인 할 수 있었습니다.

In [29]:
movie_ids = [[idx_to_movie[i[0]],i[1]] for i in similar_movie]

movie_ids = movie_ids[1:]

[[movie_ids[i][0],movie_ids[i][1]] for i in range(0,len(movie_ids))]

[['Star Trek VI: The Undiscovered Country (1991)', 0.9025383],
 ['Star Trek: Generations (1994)', 0.88010925],
 ['Star Trek: Insurrection (1998)', 0.8279797],
 ['Star Trek IV: The Voyage Home (1986)', 0.7756079],
 ['Star Trek III: The Search for Spock (1984)', 0.70790344],
 ['Star Trek V: The Final Frontier (1989)', 0.65918434],
 ['Star Trek: The Wrath of Khan (1982)', 0.64518994],
 ['Star Trek: The Motion Picture (1979)', 0.6339021],
 ['Stargate (1994)', 0.5946121],
 ['Fifth Element, The (1997)', 0.50578594],
 ['Starship Troopers (1997)', 0.46366918],
 ['X-Files: Fight the Future, The (1998)', 0.43473464],
 ['Total Recall (1990)', 0.42972943],
 ['Soft Toilet Seats (1999)', 0.41953653],
 ['Contact (1997)', 0.41886035]]

      위의 경험을 토대로 IDX와 제목을 변환해주는 코드를 작성해줍니다. 순위를 나타낼 때 자기자신은 제외하고 나오도록 작성했습니다.

In [30]:
def get_similar_movie_from_name(movie:str,N=15):
    movie_id = movie_to_idx[movie]
    similar_movie = als_model.similar_items(movie_id, N+1)
    similar_movie = [[idx_to_movie[i[0]],i[1]] for i in similar_movie]
    similar_movie = similar_movie[1:]
    
    return similar_movie

def get_similar_movie_from_idx(movie_id:int,N=15):
    print(idx_to_movie[movie_id])
    similar_movie = als_model.similar_items(movie_id, N+1)
    similar_movie = [[idx_to_movie[i[0]],i[1]] for i in similar_movie]
    similar_movie = similar_movie[1:]
    
    return similar_movie

In [31]:
get_similar_movie_from_name('Fifth Element, The (1997)')

[['Stargate (1994)', 0.67718005],
 ['Face/Off (1997)', 0.6272104],
 ['Gattaca (1997)', 0.6058121],
 ['Strange Days (1995)', 0.6032737],
 ['Total Recall (1990)', 0.5872202],
 ['Starship Troopers (1997)', 0.58650684],
 ['Dark City (1998)', 0.54892224],
 ['Mars Attacks! (1996)', 0.5444699],
 ['Matrix, The (1999)', 0.538372],
 ['Independence Day (ID4) (1996)', 0.5361375],
 ['Twelve Monkeys (1995)', 0.52018464],
 ['Star Trek: First Contact (1996)', 0.50578594],
 ['Contact (1997)', 0.50037307],
 ['Demolition Man (1993)', 0.5001871],
 ['Deep Impact (1998)', 0.48669207]]

      제5원소의 유사한 영화로 페이스오프가 나온건 의외입니다. 그 외에는 대부분 외계인, 우주선, SF가 장르가 주로 보입니다.

In [32]:
get_similar_movie_from_idx(316)

Misery (1990)


[['Scream (1996)', 0.66883075],
 ["Jacob's Ladder (1990)", 0.5975624],
 ['Shining, The (1980)', 0.5764125],
 ['Carrie (1976)', 0.57216007],
 ['Nightmare on Elm Street, A (1984)', 0.5496004],
 ['Halloween (1978)', 0.5393939],
 ['Interview with the Vampire (1994)', 0.53837854],
 ['Needful Things (1993)', 0.5052303],
 ['Poltergeist (1982)', 0.50322264],
 ['Candyman (1992)', 0.501338],
 ['Wolf (1994)', 0.49882925],
 ['Exorcist, The (1973)', 0.48981372],
 ['Omen, The (1976)', 0.48746628],
 ['Gremlins (1984)', 0.4830642],
 ["Bram Stoker's Dracula (1992)", 0.47306302]]

      임의의 숫자를 넣었는데 미저리였군요. 확인 결과 역시 슬래셔 또는 호러 영화가 주로 비슷하다고 나왔습니다.

# 8. 좋아할 만한 영화 추천받기

      SF 영화 5개에 5점을 줘서 입력해준 'mine'을 이용하여 해당 모델의 추천을 받아보겠습니다.

In [33]:
user = user_to_idx['mine']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)

[[idx_to_movie[i[0]],i[1]] for i in movie_recommended]

[['Star Wars: Episode I - The Phantom Menace (1999)', 0.4765836],
 ['Raiders of the Lost Ark (1981)', 0.47358608],
 ['Princess Bride, The (1987)', 0.41284764],
 ['Star Trek IV: The Voyage Home (1986)', 0.3700028],
 ['Star Trek: First Contact (1996)', 0.35488272],
 ['Star Trek: Generations (1994)', 0.31789857],
 ['Back to the Future (1985)', 0.29966107],
 ['Star Trek III: The Search for Spock (1984)', 0.29016477],
 ['Star Trek VI: The Undiscovered Country (1991)', 0.2830228],
 ['Terminator, The (1984)', 0.26793113],
 ['Indiana Jones and the Last Crusade (1989)', 0.2624995],
 ['Star Trek: Insurrection (1998)', 0.2384617],
 ['Alien (1979)', 0.23173246],
 ['Aliens (1986)', 0.23054565],
 ['Superman (1978)', 0.22409526],
 ['Star Trek: The Motion Picture (1979)', 0.2171332],
 ['Matrix, The (1999)', 0.20942748],
 ['E.T. the Extra-Terrestrial (1982)', 0.20325908],
 ['Star Trek V: The Final Frontier (1989)', 0.20113364],
 ["One Flew Over the Cuckoo's Nest (1975)", 0.20037377]]

      거의 다 SF를 추천하고 있는 것을 보면 제대로 학습이 된 것 같습니다.
      
      스타워즈1을 통해 어떻게 추천을 했는지 확인해봅시다.

In [34]:
star_wars_I = movie_to_idx['Star Wars: Episode I - The Phantom Menace (1999)']
explain = als_model.explain(user, csr_data, itemid=star_wars_I)

In [35]:
[[idx_to_movie[explain[1][i][0]], explain[1][i][1]] for i in range(0, len(explain[1]))]

[['Star Wars: Episode VI - Return of the Jedi (1983)', 0.22908096152418728],
 ['Star Wars: Episode IV - A New Hope (1977)', 0.11780052713130644],
 ['Stargate (1994)', 0.10308496907921474],
 ['Star Wars: Episode V - The Empire Strikes Back (1980)',
  0.06844235603954292],
 ['Star Trek: The Wrath of Khan (1982)', -0.0478247461046907]]

      스타트렉을 제외하고는 전부 매우 긍정적인 점수를 제공했습니다.

***********************************************************************************
# 총평 및 결론

      약 0.2에서 거의 0.5 사이에서 추천영화가 나타났는데 생각보다 정확도가 높은것같다,
      
      원래 작성하던 코드는 IDX에서 movie_ID로, movie_ID에서 title로 변환해주는 코드를 작성했는데 프로젝트를 진행할수록 코드가 너무 꼬이고 번잡해지는것 같아서 Merge를 통해 두 데이터셋을 합쳐주었습니다. 이 경우 CSR파일을 만들 때 제목이 그대로 문자열로 들어가 오류를 발생하는 것 같아서 결국 IDX로 변환해주었습니다.
      
      Merge를 하는 경우에도 Index나 없는 user_ID땜에 오류를 발생하는 것 같아서 다 제거하고 진행하는데 생각보다 과정이 복잡해서 시간을 많이 잡아먹게 되었습니다.